In [1]:
from argparse import Namespace
import pickle
import sys
from os import getcwd
from os.path import join
from pandas import DataFrame
import re
from fuzzywuzzy import fuzz
import jellyfish
import itertools
import numpy as np

sys.path.insert(0, join(getcwd(), "../module_code"))

from data.load import load_data, load_outcomes
from data.utils import read_files_and_combine
from cli_utils import load_cli_args, init_cli_args

sys.argv = [sys.argv[0]]
load_cli_args("../options.yml")
args = init_cli_args()

/home/jeffreyfeng/mamba/mamba/envs/crrtenv/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/home/jeffreyfeng/mamba/mamba/envs/crrtenv/lib/python3.9/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/jeffreyfeng/mamba/mamba/envs/crrtenv/lib/python3.9/site-packages/shap/utils/_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument wa

# Current mappings

In [3]:
with open("../../Data/UCLA_v3/Medications_Mapping.pkl", "rb") as f:
    med_mapping = pickle.load(f)
med_mapping

{'AZITHROMYCIN': 'MACROLIDE ANTIBIOTICS',
 'ERYTHROMYCINS': 'MACROLIDE ANTIBIOTICS',
 'CLARITHROMYCIN': 'MACROLIDE ANTIBIOTICS'}

In [4]:
with open("../../Data/Controls_v3/Medications_Mapping.pkl", "rb") as f:
    med_mapping = pickle.load(f)
med_mapping

{'AZITHROMYCIN': 'MACROLIDE ANTIBIOTICS',
 'ERYTHROMYCINS': 'MACROLIDE ANTIBIOTICS',
 'CLARITHROMYCIN': 'MACROLIDE ANTIBIOTICS'}

In [5]:
with open("../../Data/Cedars_v3/Medications_Mapping.pkl", "rb") as f:
    med_mapping = pickle.load(f)
med_mapping

{'ANTIARRHYTHMIC - CLASS III': 'ANTIARRHYTHMICS TYPE III',
 'ANTIDIURETIC AND VASOPRESSOR HORMONES': 'VASOPRESSORS',
 'ANALGESIC OPIOID AGONISTS': 'OPIOID AGONISTS',
 'CARDIOVASCULAR SYMPATHOMIMETICS': 'SYMPATHOMIMETICS',
 'MINERALS AND ELECTROLYTES - POTASSIUM FOR INJECTION': 'POTASSIUM',
 'ANTIPROTOZOAL-ANTIBACTERIAL 1ST GENERATION 2-METHYL-5-NITROIMIDAZOLE': 'ANTIPROTOZOAL AGENTS',
 'HUMAN INSULINS - SHORT ACTING': 'INSULIN',
 'PARENTERAL NUTRITION - INTRAVENOUS FAT EMULSIONS': 'LIPIDS',
 'ANALGESIC OR ANTIPYRETIC NON-OPIOID': 'ANALGESICS OTHER',
 'ARTIFICIAL TEARS AND LUBRICANT SINGLE AGENTS': 'ARTIFICIAL TEARS AND LUBRICANTS',
 'ANTIANGINAL - CORONARY VASODILATORS (NITRATES)': 'NITRATES',
 'SALICYLATE ANALGESICS': 'SALICYLATE',
 'BETA BLOCKERS CARDIAC SELECTIVE': 'BETA BLOCKERS CARDIO-SELECTIVE',
 'GALLSTONE SOLUBILIZING (LITHOLYSIS) AGENTS': 'GALLSTONE SOLUBILIZING AGENTS',
 'ANTIFUNGAL - GLUCAN SYNTHESIS INHIBITOR, ECHINOCANDINS': 'ANTIFUNGAL - GLUCAN SYNTHESIS INHIBITORS',
 'IN

In [6]:
with open("../../Data/UCLA_v3/Labs_Mapping.pkl", "rb") as f:
    lab_mapping = pickle.load(f)
lab_mapping

{'GLUCOSE,POC': 'GLUCOSE',
 'BICARBONATE': 'BICARBONATE, ARTERIAL$BICARBONATE, VENOUS',
 'BASE EXCESS': 'BASE EXCESS, ARTERIAL$BASE EXCESS, VENOUS',
 'O2 SATURATION/MEASURED': 'O2 SATURATION-ARTERIAL$O2 SATURATION-VENOUS',
 'PH': 'PH, ARTERIAL$PH,VENOUS',
 'PCO2': 'PCO2, ARTERIAL$PCO2,VENOUS',
 'PO2': 'PO2, ARTERIAL$PO2,VENOUS',
 'GLUCOSE, POINT OF CARE': 'GLUCOSE',
 'ABG INSPIRED O2': 'FIO2, ARTERIAL',
 'GLOMERULAR FILTRATION RATE': 'GFR ESTIMATE',
 'POTASSIUM,POC': 'POTASSIUM',
 'PCO2, ARTERIAL,POC': 'PCO2, ARTERIAL',
 'PH, ARTERIAL,POC': 'PH, ARTERIAL',
 'BICARBONATE, ARTERIAL,POC': 'BICARBONATE, ARTERIAL',
 'PO2, ARTERIAL,POC': 'PO2, ARTERIAL',
 'BASE EXCESS, ARTERIAL,POC': 'BASE EXCESS, ARTERIAL',
 'GFR EST.FOR NON-AFRICAN AMERIC': 'GFR ESTIMATE FOR NON-AFRICAN AMERICAN',
 'GFR EST.FOR AFRICAN AMERICAN': 'GFR ESTIMATE FOR AFRICAN AMERICAN',
 'HEMOGLOBIN,POC': 'HEMOGLOBIN',
 'CARBOXYHEMOGLOBIN,POC': 'CARBOXYHEMOGLOBIN',
 'METHEMOGLOBIN,POC': 'METHEMOGLOBIN',
 'OXYHEMOGLOBIN,POC': '

In [7]:
with open("../../Data/Controls_v3/Labs_Mapping.pkl", "rb") as f:
    lab_mapping = pickle.load(f)
lab_mapping

{'GLUCOSE,POC': 'GLUCOSE',
 'BICARBONATE': 'BICARBONATE, ARTERIAL$BICARBONATE, VENOUS',
 'BASE EXCESS': 'BASE EXCESS, ARTERIAL$BASE EXCESS, VENOUS',
 'O2 SATURATION/MEASURED': 'O2 SATURATION-ARTERIAL$O2 SATURATION-VENOUS',
 'PH': 'PH, ARTERIAL$PH,VENOUS',
 'PCO2': 'PCO2, ARTERIAL$PCO2,VENOUS',
 'PO2': 'PO2, ARTERIAL$PO2,VENOUS',
 'GLUCOSE, POINT OF CARE': 'GLUCOSE',
 'ABG INSPIRED O2': 'FIO2, ARTERIAL',
 'GLOMERULAR FILTRATION RATE': 'GFR ESTIMATE',
 'POTASSIUM,POC': 'POTASSIUM',
 'PCO2, ARTERIAL,POC': 'PCO2, ARTERIAL',
 'PH, ARTERIAL,POC': 'PH, ARTERIAL',
 'BICARBONATE, ARTERIAL,POC': 'BICARBONATE, ARTERIAL',
 'PO2, ARTERIAL,POC': 'PO2, ARTERIAL',
 'BASE EXCESS, ARTERIAL,POC': 'BASE EXCESS, ARTERIAL',
 'GFR EST.FOR NON-AFRICAN AMERIC': 'GFR ESTIMATE FOR NON-AFRICAN AMERICAN',
 'GFR EST.FOR AFRICAN AMERICAN': 'GFR ESTIMATE FOR AFRICAN AMERICAN',
 'HEMOGLOBIN,POC': 'HEMOGLOBIN',
 'CARBOXYHEMOGLOBIN,POC': 'CARBOXYHEMOGLOBIN',
 'METHEMOGLOBIN,POC': 'METHEMOGLOBIN',
 'OXYHEMOGLOBIN,POC': '

In [8]:
with open("../../Data/Cedars_v3/Labs_Mapping.pkl", "rb") as f:
    lab_mapping = pickle.load(f)
lab_mapping

{'GLUCOSE-POC': 'GLUCOSE',
 'BLOOD UREA NITROGEN': 'UREA NITROGEN',
 'SERUM CREATININE': 'CREATININE',
 'CO2': 'TOTAL CO2',
 'GLUCOSE RANDOM': 'GLUCOSE',
 'PLATELET COUNT': 'PLATELET COUNT, AUTO',
 'RED CELL DISTRIBUTION WIDTH': 'RED CELL DISTRIBUTION WIDTH-CV',
 'MEAN CORPUSCULAR HEMOGLOBIN CONC': 'MCH CONCENTRATION',
 'FREE CALCIUM': 'IONIZED CA++,CORRECTED',
 'FIO2-ARTERIAL': 'FIO2, ARTERIAL',
 'PH-ARTERIAL': 'PH, ARTERIAL',
 'PCO2-ARTERIAL': 'PCO2, ARTERIAL',
 'PO2-ARTERIAL': 'PO2, ARTERIAL',
 'HCO3-ARTERIAL': 'BICARBONATE, ARTERIAL',
 'BASE EXCESS-ARTERIAL': 'BASE EXCESS, ARTERIAL',
 'PATIENT PROTIME': 'PROTHROMBIN TIME',
 'SERUM ALBUMIN': 'ALBUMIN',
 'BILIRUBIN TOTAL': 'BILIRUBIN,TOTAL',
 'LYMPHOCYTES': 'LYMPHOCYTE PERCENT, AUTO',
 'MONOCYTES': 'MONOCYTE PERCENT, AUTO',
 'BASOPHILS': 'BASOPHIL PERCENT, AUTO',
 'EGFR NON-AFRICAN AMERICAN': 'GFR ESTIMATE FOR NON-AFRICAN AMERICAN',
 'EGFR AFRICAN AMERICAN': 'GFR ESTIMATE FOR AFRICAN AMERICAN',
 'L-LACTATE': 'BLOOD LACTATE',
 'BILIRU

In [9]:
with open("../../Data/Cedars_v3/unit_mappings.pkl", "rb") as f:
    lab_mapping = pickle.load(f)
lab_mapping

{'mode_unit': {' O2 SATURATION/MEASURED,ECMO PREMEMBRANE': '%',
  '% CD16+CD56 (NK CELLS)': '%',
  '% CD19 (B CELLS)': '%',
  '% CD19 (B CELLS)(QST)': '%',
  '% CD3 (MATURE T CELLS)(QST)': '%',
  '% CD3 (T CELLS)': '%',
  '% CD3+ OF TOTAL NUCLEATED CELLS': '%',
  '% CD4 (HELPER CELLS)': '%',
  '% CD4 (HELPER CELLS)(QST)': '%',
  "% CD8 (SUPP'R CELLS)": '%',
  '% CD8 (SUPPRESSOR T CELLS)(QST)': '%',
  '% CYTO E:T 100': '%',
  '% CYTO E:T 12.5': '%',
  '% CYTO E:T 25': '%',
  '% CYTO E:T 3.13': '%',
  '% CYTO E:T 50': '%',
  '% CYTO E:T 6.25': '%',
  '% FREE PSA(LBC)': '%',
  '% FREE PSA(QST)': '% (calc)',
  '% MONOMERIC PROLACTIN': '%',
  '% SATURATION': '%',
  '% SATURATION(QST)': '% (calc)',
  '%OXIDATION POSITIVE NEUTROPHIL': '%',
  '%SATURATION(LDQ)': '%',
  '(1,3)-BETA-D-GLUCAN': 'pg/mL',
  '(LD) FRACTION 1(LBC)': '%',
  '(LD) FRACTION 2(LBC)': '%',
  '(LD) FRACTION 3(LBC)': '%',
  '(LD) FRACTION 4(LBC)': '%',
  '(LD) FRACTION 5(LBC)': '%',
  '0.30% NACL': '%',
  '0.35% NACL': '%',

In [17]:
with open("../../Data/Cedars_v3/Procedures_Code_Mapping.pkl", "rb") as f:
    lab_mapping = pickle.load(f)
lab_mapping

{'IMG1853': '72220',
 'IMG1902': 'na',
 'IMG2125': 'na',
 'IMG4562': 'na',
 'IMG5380': '73562',
 'IMG5392': '71101',
 'IMG5394': '73010',
 'IMG5395': '73020',
 'IMG5396': '73020',
 'IMG5404': '73590',
 'IMG5412': '73100',
 'IMG5414': '73110',
 'IMG5417': 'na',
 'IMG5418': '73600',
 'IMG5420': '73610',
 'IMG5421': '73610',
 'IMG5433': '71021',
 'IMG5439': '73070',
 'IMG5440': '73080',
 'IMG5441': '73080',
 'IMG5442': '73550',
 'IMG5443': '73550',
 'IMG5451': '73630',
 'IMG5452': '73620',
 'IMG5453': '73620',
 'IMG5458': '73120',
 'IMG5460': '73130',
 'LAB456': '82009',
 'IMG5467': '73510',
 'IMG324': 'na',
 'IMG1149': 'na',
 'IVT10': 'na',
 'IMG1157': 'na',
 'IMG1162': 'na',
 'LAB395': '82947',
 'LAB1291': '87073',
 'LAB1293': '87088',
 'LAB711': '82533',
 'LAB736': '83887',
 'LAB877': '87116',
 'IMG1168': 'na',
 'IMG1169': 'na',
 'IMG1170': 'na',
 'IMG1198': 'na',
 'IMG1241': 'na',
 'PRO95': 'na',
 'IMG1007': 'na',
 'IMG1133': 'na',
 'IMG2276': 'na',
 'IMG2809': 'na',
 'IMG2811': 'na',

In [6]:
[lab for lab in list(lab_mapping.keys()) if "O2" in lab]

['FIO2-ARTERIAL',
 'PCO2-ARTERIAL',
 'PO2-ARTERIAL',
 'PO2-VENOUS',
 'FIO2-VENOUS',
 'PCO2-VENOUS',
 'FIO2']

# Look at meds

In [ ]:
cedars_meds = read_files_and_combine(['Medications.txt'], args.cedars_crrt_data_dir)
ctrl_meds = read_files_and_combine(['Medications.txt'], args.ucla_control_data_dir)
ucla_meds = read_files_and_combine(['Medications.txt'], args.ucla_crrt_data_dir)

ctrl_meds = ctrl_meds.rename(
    {
        "MEDISPAN_SUBCLASS_NAME": "PHARM_SUBCLASS",
        "ORDERING_DATE": "ORDER_DATE",
        "NAME": "MEDICATION_NAME",
    },
    axis=1,
)

# one case
ctrl_meds = ctrl_meds.replace(
    {
        "PHARM_SUBCLASS": {" ": "INSULIN"}
    }
)

cedars_meds = cedars_meds.dropna(subset=["PHARM_SUBCLASS"])
ucla_meds = ucla_meds.dropna(subset=["PHARM_SUBCLASS"])
ctrl_meds = ctrl_meds.dropna(subset=["PHARM_SUBCLASS"])

for column in  ['PHARM_SUBCLASS']:
    ucla_meds[column] = ucla_meds[column].str.upper()
for column in  ['PHARM_SUBCLASS']:
    cedars_meds[column] = cedars_meds[column].str.upper()
for column in  ['PHARM_SUBCLASS']:
    ctrl_meds[column] = ctrl_meds[column].str.upper()

ucla_meds = ucla_meds[~ucla_meds['PHARM_SUBCLASS'].str.contains('EACH')] #277
ucla_meds = ucla_meds[~ucla_meds['PHARM_SUBCLASS'].str.isnumeric()] # 251
ctrl_meds = ctrl_meds[~ctrl_meds['PHARM_SUBCLASS'].str.contains('EACH')] #277
ctrl_meds = ctrl_meds[~ctrl_meds['PHARM_SUBCLASS'].str.isnumeric()] # 251

Columns (7,8,9,10,18) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.


In [146]:
only_cedars_meds = (set(cedars_meds['PHARM_SUBCLASS'].unique())).difference(set(ucla_meds['PHARM_SUBCLASS'].unique()))
all_ucla_meds = set(ucla_meds['PHARM_SUBCLASS'].unique())
print("Total Cedars Rows: ", len(cedars_meds))
print("Total UCLA Rows: ", len(ucla_meds))

# print("Total Cedars Rows Remapped: ", len(cedars_meds[cedars_meds['PHARM_SUBCLASS'].isin(mapping.keys())]))
# print("Total UCLA Rows Corresponding to Mapping: ", len(ucla_meds[ucla_meds['PHARM_SUBCLASS'].isin(mapping.values())]))

Total Cedars Rows:  787635
Total UCLA Rows:  3432450


In [50]:
unique_ucla_meds = ucla_meds['PHARM_SUBCLASS'].value_counts().index.to_list()
for item in unique_ucla_meds:
    print(item)

INSULIN
OPIOID AGONISTS
IMMUNOSUPPRESSIVE AGENTS
PROTON PUMP INHIBITORS
SODIUM
HEPARINS AND HEPARINOID-LIKE AGENTS
GLUCOCORTICOSTEROIDS
VASOPRESSORS
CONTINUOUS RENAL REPLACEMENT THERAPY (CRRT) SOLUTIONS
ANESTHETICS - MISC.
LOOP DIURETICS
LAXATIVES - MISCELLANEOUS
MAGNESIUM
ANTISEPTICS - MOUTH/THROAT
WATER SOLUBLE VITAMINS
SURFACTANT LAXATIVES
PLASMA PROTEINS
B-COMPLEX W/ FOLIC ACID
GALLSTONE SOLUBILIZING AGENTS
ANTI-INFECTIVE AGENTS - MISC.
ANTICONVULSANTS - MISC.
IMIDAZOLE-RELATED ANTIFUNGALS
ANALGESICS OTHER
ALTERNATIVE MEDICINE - M'S
POTASSIUM
NON-BARBITURATE HYPNOTICS
5-HT3 RECEPTOR ANTAGONISTS
PENICILLIN COMBINATIONS
SALICYLATES
SYMPATHOMIMETICS
HMG COA REDUCTASE INHIBITORS
OIL SOLUBLE VITAMINS
LOCAL ANESTHETICS - TOPICAL
DIBENZAPINES
THYROID HORMONES
CARBOHYDRATES
STIMULANT LAXATIVES
BETA BLOCKERS CARDIO-SELECTIVE
ANTI-INFECTIVE MISC. - COMBINATIONS
GLYCOPEPTIDES
LIPIDS
BRONCHODILATORS - ANTICHOLINERGICS
HEMATOPOIETIC GROWTH FACTORS
PHOSPHATE BINDER AGENTS
ZINC
CALCIUM CHANNEL BL

In [56]:
for item in ctrl_meds['PHARM_SUBCLASS'].value_counts().index.to_list():
    if item in unique_ucla_meds:
        print(item,'$',item)
    else:
        print(item,'$')

INSULIN $ INSULIN
HEPARINS AND HEPARINOID-LIKE AGENTS $ HEPARINS AND HEPARINOID-LIKE AGENTS
OPIOID AGONISTS $ OPIOID AGONISTS
CONTINUOUS RENAL REPLACEMENT THERAPY (CRRT) SOLUTIONS $ CONTINUOUS RENAL REPLACEMENT THERAPY (CRRT) SOLUTIONS
IMMUNOSUPPRESSIVE AGENTS $ IMMUNOSUPPRESSIVE AGENTS
VASOPRESSORS $ VASOPRESSORS
ANESTHETICS - MISC. $ ANESTHETICS - MISC.
SODIUM $ SODIUM
PROTON PUMP INHIBITORS $ PROTON PUMP INHIBITORS
NON-BARBITURATE HYPNOTICS $ NON-BARBITURATE HYPNOTICS
GLUCOCORTICOSTEROIDS $ GLUCOCORTICOSTEROIDS
SYMPATHOMIMETICS $ SYMPATHOMIMETICS
GALLSTONE SOLUBILIZING AGENTS $ GALLSTONE SOLUBILIZING AGENTS
LOOP DIURETICS $ LOOP DIURETICS
ANTICONVULSANTS - MISC. $ ANTICONVULSANTS - MISC.
LAXATIVES - MISCELLANEOUS $ LAXATIVES - MISCELLANEOUS
INOTROPES $ INOTROPES
SURFACTANT LAXATIVES $ SURFACTANT LAXATIVES
ANTI-INFECTIVE AGENTS - MISC. $ ANTI-INFECTIVE AGENTS - MISC.
PENICILLIN COMBINATIONS $ PENICILLIN COMBINATIONS
ANTISEPTICS - MOUTH/THROAT $ ANTISEPTICS - MOUTH/THROAT
WATER SOLUBL

In [57]:
for item in cedars_meds['PHARM_SUBCLASS'].value_counts().index.to_list():
    if item in unique_ucla_meds:
        print(item,'$',item)
    else:
        print(item,'$')

ANTIARRHYTHMIC - CLASS III $
ANTIDIURETIC AND VASOPRESSOR HORMONES $
ANALGESIC OPIOID AGONISTS $
CARDIOVASCULAR SYMPATHOMIMETICS $
MINERALS AND ELECTROLYTES - POTASSIUM FOR INJECTION $
ANTIPROTOZOAL-ANTIBACTERIAL 1ST GENERATION 2-METHYL-5-NITROIMIDAZOLE $
HUMAN INSULINS - SHORT ACTING $
PARENTERAL NUTRITION - INTRAVENOUS FAT EMULSIONS $
ANALGESIC OR ANTIPYRETIC NON-OPIOID $
ARTIFICIAL TEARS AND LUBRICANT SINGLE AGENTS $
ANTIANGINAL - CORONARY VASODILATORS (NITRATES) $
SALICYLATE ANALGESICS $
BETA BLOCKERS CARDIAC SELECTIVE $
GALLSTONE SOLUBILIZING (LITHOLYSIS) AGENTS $
ANTIFUNGAL - GLUCAN SYNTHESIS INHIBITOR, ECHINOCANDINS $
INSULIN ANALOGS - RAPID ACTING $
ARTIFICIAL TEARS AND LUBRICANT COMBINATIONS $
MINERALS AND ELECTROLYTES - ZINC $
ANTICONVULSANT - PYRROLIDINE DERIVATIVES $
MINERALS AND ELECTROLYTES - CALCIUM REPLACEMENT $
HEMODIALYSIS AND HEMOFILTRATE SOLUTIONS $
RINGER'S AND LACTATED RINGER'S SOLUTIONS $
ANTIPSYCHOTIC -ATYPICAL DOPAMINE-SEROTONIN ANTAG-DIBENZOTHIAZEPINE DER $
AN

# Look at labs

In [2]:
from data.longitudinal_features import map_labs

labs = "Labs.txt"

crrt_df = read_files_and_combine([labs], args.ucla_crrt_data_dir)
ctrl_df = read_files_and_combine([labs], args.ucla_control_data_dir)
cedars_df = read_files_and_combine([labs], args.cedars_crrt_data_dir).rename(
    {"RESULT": "RESULTS", "NAME": "COMPONENT_NAME"}, axis=1
)
# cedars_df = map_labs(cedars_df, args.cedars_crrt_data_dir)

mealy | WARNING - Unexpected encoding in Labs.txt. Encoding with cp1252.


In [3]:
print(len(crrt_df))
print(len(ctrl_df))
print(len(cedars_df))

16652634
21855564
810449


In [68]:
print(crrt_df['COMPONENT_NAME'].value_counts().to_string())

GLUCOSE,POC                                                              565199
GLUCOSE                                                                  320040
SODIUM                                                                   310425
POTASSIUM                                                                309890
GFR ADDITIONAL INFORMATION                                               305903
CHLORIDE                                                                 304416
CREATININE                                                               304354
TOTAL CO2                                                                304288
UREA NITROGEN                                                            303725
CALCIUM                                                                  303114
ANION GAP                                                                300588
HEMOGLOBIN                                                               292200
GFR ESTIMATE FOR NON-AFRICAN AMERICAN   

In [43]:
unique_ucla_labs = crrt_df['COMPONENT_NAME'].value_counts().index.to_list()
for item in unique_ucla_labs:
    print(item)

GLUCOSE,POC
GLUCOSE
SODIUM
POTASSIUM
GFR ADDITIONAL INFORMATION
CHLORIDE
CREATININE
TOTAL CO2
UREA NITROGEN
CALCIUM
ANION GAP
HEMOGLOBIN
GFR ESTIMATE FOR NON-AFRICAN AMERICAN
GFR ESTIMATE FOR AFRICAN AMERICAN
PLATELET COUNT, AUTO
HEMATOCRIT
WHITE BLOOD CELL COUNT
MEAN CORPUSCULAR VOLUME
RED BLOOD CELL COUNT
MCH CONCENTRATION
MEAN CORPUSCULAR HEMOGLOBIN
RED CELL DISTRIBUTION WIDTH-CV
RED CELL DISTRIBUTION WIDTH-SD
ABSOLUTE NUCLEATED RBC COUNT
NUCLEATED RBC%, AUTOMATED
MAGNESIUM
MEAN PLATELET VOLUME
IONIZED CA++,UNCORRECTED
PHOSPHORUS
IONIZED CA++,CORRECTED
BICARBONATE
BASE EXCESS
O2 SATURATION/MEASURED
PROTHROMBIN TIME
INR
APTT
PH
PCO2
PO2
GLUCOSE, POINT OF CARE
ALKALINE PHOSPHATASE
BILIRUBIN,TOTAL
TOTAL PROTEIN
ALANINE AMINOTRANSFERASE
ASPARTATE AMINOTRANSFERASE
ABSOLUTE MONO COUNT
ABSOLUTE LYMPHOCYTE COUNT
ABSOLUTE BASO COUNT
ABSOLUTE EOS COUNT
ABSOLUTE NEUT COUNT
LYMPHOCYTE PERCENT, AUTO
BASOPHIL PERCENT, AUTO
MONOCYTE PERCENT, AUTO
EOSINOPHIL PERCENT, AUTO
NEUTROPHIL PERCENT, AUTO
A

In [70]:
print(ctrl_df['COMPONENT_NAME'].value_counts().to_string())

GLUCOSE,POC                                                              595759
GLUCOSE                                                                  424415
HEMOGLOBIN                                                               397531
POTASSIUM                                                                394941
SODIUM                                                                   394743
CREATININE                                                               389535
CHLORIDE                                                                 388120
TOTAL CO2                                                                388040
UREA NITROGEN                                                            386309
HEMATOCRIT                                                               384098
PLATELET COUNT, AUTO                                                     383860
WHITE BLOOD CELL COUNT                                                   380871
MEAN CORPUSCULAR VOLUME                 

In [64]:
pairs = {'left': [], 'right': []}
for item in ctrl_df['COMPONENT_NAME'].value_counts().index.to_list():
    pairs['left'].append(item)
    if item in unique_ucla_labs:
        pairs['right'].append(item)
    else:
        pairs['right'].append(np.nan)
pd.DataFrame(pairs).to_csv('./temp.csv')

In [53]:
print(cedars_df['COMPONENT_NAME'].value_counts().to_string())

GLUCOSE-POC                                                      30339
BLOOD UREA NITROGEN                                              15719
CALCIUM                                                          15665
SODIUM                                                           15619
POTASSIUM                                                        15505
SERUM CREATININE                                                 15476
CHLORIDE                                                         15434
CO2                                                              15434
GLUCOSE RANDOM                                                   15427
ANION GAP                                                        15368
PLATELET COUNT                                                   14436
HEMATOCRIT                                                       14409
HEMOGLOBIN                                                       14406
RED CELL DISTRIBUTION WIDTH                                      14347
WHITE 

In [66]:
pairs = {'left': [], 'right': []}
for item in cedars_df['COMPONENT_NAME'].value_counts().index.to_list():
    pairs['left'].append(item)
    if item in unique_ucla_labs:
        pairs['right'].append(item)
    else:
        pairs['right'].append(np.nan)
pd.DataFrame(pairs).to_csv('./temp.csv')

In [6]:
cedars_df[cedars_df['COMPONENT_NAME']=='OXYGEN CONTENT, ARTERIAL']

,IP_PATIENT_ID,IP_ENCOUNTER_ID,COMPONENT_ID,COMPONENT_NAME,SPECIMN_TAKEN_TIME,PROC_ID,DESCRIPTION,ORDER_TIME,RESULT_TIME,RESULTS,REFERENCE_UNIT,LOINC_CODE
6474,2.745897e+10,93829960715,3472,"OXYGEN CONTENT, ARTERIAL",2016-09-16 05:00:00,3040080229,"BLOOD GAS, CO-OXIMETRY, ARTERIAL",2016-09-16 00:05:00,2016-09-16 05:19:00,11.0,%,9461.0
6806,2.745897e+10,93829960715,3472,"OXYGEN CONTENT, ARTERIAL",2016-09-15 04:20:00,3040080229,"BLOOD GAS, CO-OXIMETRY, ARTERIAL",2016-09-15 00:05:00,2016-09-15 04:47:00,13.3,%,9461.0
7403,2.745897e+10,93829960715,3472,"OXYGEN CONTENT, ARTERIAL",2016-09-17 05:30:00,3040080229,"BLOOD GAS, CO-OXIMETRY, ARTERIAL",2016-09-17 00:05:00,2016-09-17 05:46:00,11.3,%,9461.0
7478,2.745897e+10,93829960715,3472,"OXYGEN CONTENT, ARTERIAL",2016-09-18 05:15:00,3040080229,"BLOOD GAS, CO-OXIMETRY, ARTERIAL",2016-09-18 00:05:00,2016-09-18 05:36:00,10.9,%,9461.0
31195,NaN,82885235023,3472,"OXYGEN CONTENT, ARTERIAL",2016-11-18 13:20:00,3040080229,"BLOOD GAS, CO-OXIMETRY, ARTERIAL",2016-11-18 13:00:00,2016-11-18 13:25:00,13.2,%,9461.0
...,...,...,...,...,...,...,...,...,...,...,...,...
787298,NaN,78430510430,3472,"OXYGEN CONTENT, ARTERIAL",2017-09-07 16:40:00,3040080229,"BLOOD GAS, CO-OXIMETRY, ARTERIAL",2017-09-07 16:36:00,2017-09-07 17:03:00,12.6,%,9461.0
793783,NaN,12272434280,3472,"OXYGEN CONTENT, ARTERIAL",2017-11-26 18:25:00,3040080229,"BLOOD GAS, CO-OXIMETRY, ARTERIAL",2017-11-26 18:11:00,2017-11-26 18:42:00,11.1,%,9461.0
793889,NaN,12272434280,3472,"OXYGEN CONTENT, ARTERIAL",2017-11-28 06:40:00,3040080229,"BLOOD GAS, CO-OXIMETRY, ARTERIAL",2017-11-28 00:05:00,2017-11-28 06:55:00,8.4,%,9461.0
794603,NaN,12272434280,3472,"OXYGEN CONTENT, ARTERIAL",2017-11-27 16:00:00,3040080229,"BLOOD GAS, CO-OXIMETRY, ARTERIAL",2017-11-27 00:05:00,2017-11-27 16:25:00,9.8,%,9461.0


In [5]:
crrt_df[crrt_df['COMPONENT_NAME']=='ALBUMIN']

,IP_PATIENT_ID,IP_ENCOUNTER_ID,PROC_ID,DESCRIPTION,COMPONENT_ID,COMPONENT_NAME,ORDER_TIME,RESULT_TIME,SPECIMN_TAKEN_TIME,RESULTS,REFERENCE_UNIT,LOINC_CODE
250,0A12882DF69AE860DE04174578AFA209,618BD9FC5B321B77EBDAEE5D2A7AA388,682,COMPREHENSIVE METABOLIC PANEL,3000384,ALBUMIN,04/30/2020 23:00:00,05/01/2020 06:32:00,05/01/2020 05:11:00,2.6,g/dL,NaN
264,587497B1A71C2CDD73FD5D628A116611,B49330E910C8AB0442558E95F9F69BB2,682,COMPREHENSIVE METABOLIC PANEL,3000384,ALBUMIN,10/14/2019 11:45:00,10/14/2019 17:13:00,10/14/2019 11:45:00,4.1,g/dL,NaN
329,D8BEE761DA13D0AFADEBFFA3351842B7,BCDBA3FF382C8030C07AE24342D399FB,682,COMPREHENSIVE METABOLIC PANEL,3000384,ALBUMIN,12/06/2018 21:56:00,12/06/2018 22:54:00,12/06/2018 21:59:00,2.3,g/dL,NaN
483,F0D033B3805E19380D1A0F8718D3BE45,8D3889CF050605BD0B75E887F2BCFCB4,682,COMPREHENSIVE METABOLIC PANEL,3000384,ALBUMIN,09/10/2016 22:00:00,09/11/2016 06:55:00,09/11/2016 05:44:00,2.8,g/dL,NaN
539,44E06E9F45D6349396F68751A3DF692A,9FCDD1C9D87C11848E83F5A78B278DF3,682,COMPREHENSIVE METABOLIC PANEL,3000384,ALBUMIN,03/01/2019 22:00:00,03/02/2019 05:08:00,03/02/2019 04:00:00,3.5,g/dL,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16651693,22E67257F434A26B3021B0566F7717B0,961EBD61E2287152D510B679D7D687F7,682,COMPREHENSIVE METABOLIC PANEL,3000384,ALBUMIN,02/09/2017 23:50:00,02/10/2017 01:20:00,02/10/2017 00:20:00,2.5,g/dL,NaN
16651874,2838F4BCFCFBE5AC5EB870484ACB6C0E,F357FE2511CB129DD07146A05EC51C9F,682,COMPREHENSIVE METABOLIC PANEL,3000384,ALBUMIN,03/13/2020 02:06:00,03/13/2020 09:45:00,03/13/2020 08:54:00,3.5,g/dL,NaN
16651879,983900DDB44B60F51421ED3073A247B3,4016C94B79249DFDEE4B2A1BF2105EA8,688,HEPATIC FUNCT PANEL,3000384,ALBUMIN,12/15/2016 22:00:00,12/16/2016 06:05:00,12/16/2016 04:55:00,3.3,g/dL,NaN
16652297,CE8CD480D4791708F80C841DAA6C36C7,BA2FA9496F146A004FECDB63826AC326,886,"PEP & TOTAL PROTEIN,SERUM",8001852,ALBUMIN,12/16/2015 10:50:00,12/18/2015 15:48:00,12/16/2015 10:51:00,4.9,g/dL,2862-1


In [8]:
cedars_df[cedars_df['COMPONENT_NAME']=='MEAN CORPUSCULAR HEMOGLOBIN CONC']
# MCH CONCENTRATION

,IP_PATIENT_ID,IP_ENCOUNTER_ID,COMPONENT_ID,COMPONENT_NAME,SPECIMN_TAKEN_TIME,PROC_ID,DESCRIPTION,ORDER_TIME,RESULT_TIME,RESULTS,REFERENCE_UNIT,LOINC_CODE
10,2.662223e+10,16889280578,1576932,MEAN CORPUSCULAR HEMOGLOBIN CONC,2016-07-27 06:25:00,1698,CBC,2016-07-27 01:51:00,2016-07-27 09:10:00,30.3,%,70824.0
30,2.662223e+10,16889280578,1576932,MEAN CORPUSCULAR HEMOGLOBIN CONC,2016-07-28 05:30:00,1698,CBC,2016-07-28 00:06:00,2016-07-28 07:20:00,31.5,%,70824.0
84,2.662223e+10,16889280578,1576932,MEAN CORPUSCULAR HEMOGLOBIN CONC,2016-07-26 23:27:00,10238888,CBC WITH DIFFERENTIAL,2016-07-26 21:51:00,2016-07-27 00:37:00,31.4,%,70824.0
174,2.662223e+10,16889280578,1576932,MEAN CORPUSCULAR HEMOGLOBIN CONC,2016-07-29 12:00:00,1698,CBC,2016-07-29 11:23:00,2016-07-29 13:15:00,32.2,%,70824.0
200,2.662223e+10,16889280578,1576932,MEAN CORPUSCULAR HEMOGLOBIN CONC,2016-07-30 05:45:00,1698,CBC,2016-07-30 00:05:00,2016-07-30 07:26:00,33.0,%,70824.0
...,...,...,...,...,...,...,...,...,...,...,...,...
810196,NaN,61870286282,1576932,MEAN CORPUSCULAR HEMOGLOBIN CONC,2017-10-18 05:29:00,1698,CBC,2017-10-18 05:29:00,2017-10-18 07:42:00,33.0,%,70824.0
810295,NaN,61870286282,1576932,MEAN CORPUSCULAR HEMOGLOBIN CONC,2017-10-17 05:00:00,1698,CBC,2017-10-17 05:00:00,2017-10-17 05:31:00,33.0,%,70824.0
810318,NaN,61870286282,1576932,MEAN CORPUSCULAR HEMOGLOBIN CONC,2017-10-19 06:20:00,1698,CBC,2017-10-19 06:20:00,2017-10-19 08:04:00,34.6,%,70824.0
810412,NaN,61870286282,1576932,MEAN CORPUSCULAR HEMOGLOBIN CONC,2017-10-28 06:00:00,1698,CBC,2017-10-28 00:05:00,2017-10-28 07:21:00,31.5,%,70824.0


In [15]:
crrt_df[crrt_df['COMPONENT_NAME']=='LACTATE DEHYDROGENASE']

,IP_PATIENT_ID,IP_ENCOUNTER_ID,PROC_ID,DESCRIPTION,COMPONENT_ID,COMPONENT_NAME,ORDER_TIME,RESULT_TIME,SPECIMN_TAKEN_TIME,RESULTS,REFERENCE_UNIT,LOINC_CODE
599,96C2028CEF4480A65C9BEFE3093F8678,EC0D9B57D1A98186CFA91FA3DBFE376A,840,LACTATE DEHYDROGENASE,3000397,LACTATE DEHYDROGENASE,09/07/2017 18:00:00,09/08/2017 02:35:00,09/08/2017 01:51:00,449,U/L,NaN
704,1F84D4AEE879F911596141ED46467777,1914C35646D4458F39801EECBF506D82,840,LACTATE DEHYDROGENASE,3000397,LACTATE DEHYDROGENASE,04/05/2019 22:19:00,04/06/2019 01:08:00,04/06/2019 00:17:00,181,U/L,NaN
1196,96E959B4B6AEDF754FFE44E688651089,9A47CD1136EB9377EDC446D3FE1550CC,840,LACTATE DEHYDROGENASE,3000397,LACTATE DEHYDROGENASE,10/11/2019 22:00:00,10/12/2019 04:02:00,10/12/2019 03:08:00,130,U/L,NaN
1567,384CBF7747773B3677FD20338DF59D37,EA0137A5988894A89F5E185FD570F125,840,LACTATE DEHYDROGENASE,3000397,LACTATE DEHYDROGENASE,11/06/2019 18:00:00,11/07/2019 02:44:00,11/06/2019 23:03:00,310,U/L,NaN
1890,509BA6C1F5EFF2FC0CC60E30F0C5EBD4,F86F88D21335718F3B9591A6A5AA7862,840,LACTATE DEHYDROGENASE,3000397,LACTATE DEHYDROGENASE,04/13/2017 18:00:00,04/14/2017 01:30:00,04/14/2017 00:40:00,189,U/L,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16648677,7E8846246AAA909F730FA351EFF6E508,0B8B58A7669117DAC98DC33EA9EA191B,840,LACTATE DEHYDROGENASE,3000397,LACTATE DEHYDROGENASE,12/20/2018 16:00:00,12/21/2018 00:56:00,12/20/2018 23:49:00,721,U/L,NaN
16648994,EE8A6FFC8FAC17CB7B10AEA9AE26885B,35CFB3958E721BA5DF8E07405E9B0CE6,840,LACTATE DEHYDROGENASE,3000397,LACTATE DEHYDROGENASE,12/19/2020 18:00:00,12/20/2020 00:50:00,12/20/2020 00:03:00,186,U/L,NaN
16650572,3FCBC716ACD8D1961762B7B1C5F04CF4,DA6E90373BEA011DC831E9E8FCF6723F,840,LACTATE DEHYDROGENASE,3000397,LACTATE DEHYDROGENASE,08/15/2020 04:00:00,08/15/2020 11:49:00,08/15/2020 11:18:00,588,U/L,NaN
16650835,6E2CAC3A360BAEEE868232C987FC19A3,A3C390B9E389C22F2B09D36F3DFE5F69,840,LACTATE DEHYDROGENASE,3000397,LACTATE DEHYDROGENASE,06/06/2019 13:05:00,06/06/2019 14:53:00,06/06/2019 13:27:00,176,U/L,NaN


### Creatinine example

In [49]:
ctrl_df[ctrl_df['COMPONENT_NAME'].str.contains('CREATININE')]['COMPONENT_NAME'].value_counts()

CREATININE                                                389535
CREATININE,PLASMA                                          81118
CREATININE,RANDOM URINE                                    16744
CREATININE(QST)                                             8994
BUN/CREATININE RATIO(QST)                                   8923
CREATININE (OSL)                                            5983
ALBUMIN/CREATININE RATIO                                    3405
CREATININE,WHOLE BLOOD                                      2743
CREATININE, SERUM(LBC)                                      2643
BUN/CREATININE RATIO(LBC)                                   2596
CREATININE(LDQ)                                             2237
CREATININE,ANES                                              860
CREATININE CONCENTRATION,URINE                               502
CREATININE, RANDOM URINE(QST)                                368
CREATININE, TIMED URINE                                      310
CREATININE,FLUID         

In [27]:
crrt_df[crrt_df['COMPONENT_NAME'].str.contains('CREATININE')]['COMPONENT_NAME'].value_counts()

CREATININE                                                304354
CREATININE,PLASMA                                          61589
CREATININE,RANDOM URINE                                     6795
CREATININE (OSL)                                            2789
CREATININE(QST)                                             2613
BUN/CREATININE RATIO(QST)                                   2594
CREATININE(LDQ)                                              817
CREATININE,WHOLE BLOOD                                       816
CREATININE, SERUM(LBC)                                       680
BUN/CREATININE RATIO(LBC)                                    637
ALBUMIN/CREATININE RATIO                                     591
CREATININE CONCENTRATION,URINE                               204
CREATININE,FLUID                                             160
CREATININE, TIMED URINE (NOT CLEARANCE)                      135
CREATININE, URINE - PER VOLUME                                73
CREATININE, RANDOM URINE(

In [25]:
cedars_df[cedars_df['COMPONENT_NAME'].str.contains('CREATININE')]['COMPONENT_NAME'].value_counts()

SERUM CREATININE                 15476
URINE CREATININE UCRR              386
URINE CREATININE, 24H              376
URINE CREATININE-MG/DL              38
URINE CREATININE-G/24H              32
CREATININE CLEARANCE                32
CREATININE FLUID                    22
CREATININE (POC)                    21
MICROALBUMIN/CREATININE RATIO        6
CREATININE-UMOL/L, URINE             3
URINE CREATININE UCRA                1
Name: COMPONENT_NAME, dtype: int64